In [ ]:
#an email bot that reads the emails an add them to labels according to the context and u can retrieve the emails in all labels

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import pandas as pd
import time
from datetime import datetime

In [ ]:
# ChromeDriver Path
chrome_driver_path = r"C:\Users\Syndictech\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
# chrome_options.add_argument("--headless")  # Uncomment to run headless

# Setting up the webdriver
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open Gmail
driver.get('https://mail.google.com')

# Wait for the login page to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'identifierId')))

# Login
driver.find_element(By.ID, 'identifierId').send_keys('waleed.uyi.bot@gmail.com')
driver.find_element(By.ID, 'identifierNext').click()

# Wait for the password page to load and enter the password
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'Passwd')))
    password_input = driver.find_element(By.NAME, 'Passwd')
    password_input.send_keys('UYI@waleedBOT')
    driver.find_element(By.ID, 'passwordNext').click()
except TimeoutException:
    print("Password input field was not found. Please check the locator or wait time.")
    driver.quit()
    exit()

# Wait for Gmail to load
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]')))

# Function to navigate to a label
def go_to_label(label_name):
    try:
        label_xpath = f'//a[contains(text(), "{label_name}")]'
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, label_xpath))).click()
        time.sleep(3)  # Adjust as necessary
    except TimeoutException:
        print(f"Error: Label '{label_name}' not found or took too long to load.")
    except Exception as e:
        print(f"Error navigating to label '{label_name}': {e}")

# Function to parse the date
def parse_email_date(date_str):
    try:
        date_str = date_str.replace('\u202f', ' ')
        date_obj = datetime.strptime(date_str, '%a, %b %d, %Y, %I:%M %p')
        return date_obj.strftime('%Y-%m-%d %H:%M:%S')  # Convert to a more standard format
    except ValueError as e:
        print(f"Error parsing date '{date_str}': {e}")
        return date_str  # Return the original string if parsing fails

# Updated function to extract all emails from the current label
def extract_all_emails():
    emails = []
    try:
        email_elements = driver.find_elements(By.CLASS_NAME, 'zA')

        if not email_elements:
            print("No emails found in this label.")
            return emails

        for email in email_elements:
            try:
                subject = email.find_element(By.CLASS_NAME, 'bog').text
                sender = email.find_element(By.CSS_SELECTOR, '.yX .zF, .yX .yP').get_attribute('email')
                snippet = email.find_element(By.CLASS_NAME, 'y2').text
                date_element = email.find_element(By.CSS_SELECTOR, '.xW.xY span')
                date = date_element.get_attribute('title')

                # Parsing the date string to a proper datetime object
                try:
                    email_date = datetime.strptime(date, '%b %d, %Y, %I:%M %p')
                except ValueError:
                    try:
                        email_date = datetime.strptime(date, '%b %d, %Y')
                    except ValueError:
                        email_date = date  # In case the format doesn't match expected patterns

                # Drop record if both subject and snippet are empty
                if subject.strip() == "" and snippet.strip() == "":
                    continue

                emails.append([subject, sender, snippet, email_date])
            except NoSuchElementException as e:
                print(f"Error extracting email details: {e}")
                continue

    except Exception as e:
        print(f"Error while extracting emails: {e}")

    return emails

# Function to retrieve emails from a specific label or all emails
def get_emails(label_name=None, label_names=None):
    all_emails = []

    if label_name and label_name.lower() != "all":
        # If a specific label is requested
        df_label = get_emails_from_label(label_name)
        if df_label is not None:
            df_label['Label'] = label_name
            return df_label

    elif label_names or (label_name and label_name.lower() == "all"):
        # If all labels are requested
        for label in label_names:
            df_label = get_emails_from_label(label)
            if df_label is not None:
                df_label['Label'] = label
                all_emails.append(df_label)

        if all_emails:
            return pd.concat(all_emails, ignore_index=True)
        else:
            print("No emails found in the selected labels.")
            return pd.DataFrame(columns=['Subject', 'Sender', 'Snippet', 'Date', 'Label'])

# Function to retrieve emails from a specific label
def get_emails_from_label(label_name):
    go_to_label(label_name)
    emails = extract_all_emails()
    if emails:
        return pd.DataFrame(emails, columns=['Subject', 'Sender', 'Snippet', 'Date'])
    else:
        return None
        
def move_email_to_label(driver, label_name):
    try:
        # Wait for the email checkbox to be clickable and select it
        email_checkbox = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.oZ-jc'))
        )
        
        # Click the checkbox to select the email
        is_checked = email_checkbox.get_attribute('aria-checked')
        if is_checked == 'false':
            email_checkbox.click()
            WebDriverWait(driver, 5).until(
                lambda d: email_checkbox.get_attribute('aria-checked') == 'true'
            )
        
        # Ensure the "Move to" button is visible and clickable
        move_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.T-I.J-J5-Ji.ns.T-I-ax7.T-I-Js-Gs.mA'))
        )
        move_button.click()

        # Wait for the label menu to appear
        label_menu = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.J-M.jQjAxd[role="menu"]'))
        )
        
        # Ensure the label element is visible and clickable
        label_xpath = f'//div[@class="J-N-Jz" and text()="{label_name}"]'
        label_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, label_xpath))
        )
        label_element.click()
        
        print(f"Successfully moved email to label '{label_name}'")
        
    except Exception as e:
        print(f"Error while moving email to label '{label_name}': {e}")




def categorize_and_move_unread_emails():
    try:
        go_to_label('Inbox')  # Go to the inbox to retrieve unread emails
        email_elements = driver.find_elements(By.CSS_SELECTOR, 'tr.zA.zE')  # Unread emails have the class 'zE'

        for email in email_elements:
            try:
                # Extract the sender's name from the email details
                email_details_element = email.find_element(By.CSS_SELECTOR, 'span.y2')
                email_details = email_details_element.text
                
                # Find the sender's name in the email details
                if "From:" in email_details:
                    sender_name = email_details.split("From:")[1].split(" <")[0].strip()
                else:
                    sender_name = "Unknown"

                print(f"Sender: {sender_name}")  # Debug: Print sender name

                # Click the checkbox to select the email
                checkbox = email.find_element(By.CSS_SELECTOR, 'div.oZ-jc')
                if checkbox.get_attribute('aria-checked') == 'false':
                    checkbox.click()

                # Determine the label based on the sender's name
                label_name = 'Careem' if 'Careem' in sender_name else 'Other'
                print(f"Labeling as: {label_name}")  # Debug: Print label name

                # Move the email to the determined label
                move_email_to_label(driver, label_name)
            except Exception as e:
                print(f"Error processing email: {e}")

    except Exception as e:
        print(f"Error categorizing and moving unread emails: {e}")


# Example usage:
categorize_and_move_unread_emails()

# Prompt user for the label name
label_names = ['Clients', 'Partnerships', 'Research', 'Product Development', 'Marketing', 'Finance', 'Careem', 'AI Tools','Other']

user_label_input = input(f"Labels available: {label_names}\nEnter the label you want to retrieve emails from (type 'all' to retrieve from all labels): ")

# Retrieve emails based on user input
df_emails = get_emails(label_name=user_label_input, label_names=label_names)

# Save the retrieved emails to a CSV file
if df_emails is not None and not df_emails.empty:
    if user_label_input.lower() == 'all':
        df_emails.to_csv('all_emails_new.csv', index=False)
    else:
        df_emails.to_csv(f'{user_label_input}_emails.csv', index=False)
    print(f"Emails saved to {user_label_input.lower()}_emails.csv")
else:
    print("No emails were retrieved, so no CSV file was created.")

# Close the driver
driver.quit()